## Cost of Living Among Countries Project

### Business Understanding

For this project, I was interestested in using the Global Cost of Living data from Numbeo to answer the following:

1. What are the countries with highest and lowest Salary-Cost ratio considering the following indicators?

    - Cost of a meal in an inexpensive restaurant
    - Cost of gasoline
    - Cost of basic services
    - Cost of a Square Meter to buy an apartment in City Center


2. Do countries with highest Cost of Living Index are necessarly the ones with highest purchasing power index?

3. It is possible to accurately predict the average monthly salary of a country based on their living costs?

### Data Understanding 

In the first place, we will import the necessary libraries. Then, we will access the two available datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import plotly.express as px

%matplotlib inline
# Read both data sets
df_1 = pd.read_csv('cost-of-living.csv', index_col=0)
df_2 = pd.read_csv('cost-of-living_v2.csv', index_col=0)


We will explore both datasets and determine which data set to use based on the amount of missing values in df_1 and df_2

In [ ]:
# Which dataset has more missing values?

proportion = (df_1.isna().sum()/df_2.isna().sum()).mean()
std = (df_1.isna().sum()/df_2.isna().sum()).std()

print('Data Set 1 has {:.2f} +- {:.2f} missing values with respect Data Set 2'.format(proportion, std))

We will use Data Set 1 since it has less missing values than Data Set 2. 
The column 'data_quality' of the dataframe is 0 if Numbeo considers that more contributors are needed to increase data quality, else 1. Hence, we will only keep good quality data.

In [ ]:
df = df_1[df_1['data_quality']==1]
df.head()

We rename columns for their meaning to understand further visualizations and tables. Also, we drop the data_quality column since it is 1 for all the rows and doesn't bring any information.

Then, we describe the dataset

In [ ]:
df.rename(columns = {'x1':'MealInexpensive',
                     'x2':'Meal2People',
                     'x3':'McMeal',
                     'x4':'BeerDomesticRes',
                     'x5':'BeerImportedRes',
                     'x6':'Cappuccino',
                     'x7':'Coke',
                     'x8':'Water',
                     'x9':'Milk',
                     'x10':'Bread',
                     'x11':'Rice',
                     'x12':'Eggs',
                     'x13':'Cheese',
                     'x14':'Chicken',
                     'x15':'Beef',
                     'x16':'Apples',
                     'x17':'Banana',
                     'x18':'Oranges',
                     'x19':'Tomato',
                     'x20':'Potato',
                     'x21':'Onion',
                     'x22':'Lettuce',
                     'x23':'WaterM',
                     'x24':'Wine',
                     'x25':'BeerDomesticMarket',
                     'x26':'BeerImportedMarket',
                     'x27':'Cigarettes',
                     'x28':'OWTicket',
                     'x29':'MonthlyPass',
                     'x30':'TaxiStart',
                     'x31':'Taxi1km',
                     'x32':'Taxi1h',
                     'x33':'Gasoline',
                     'x34':'VWGolf',
                     'x35':'Corolla',
                     'x36':'Services',
                     'x37':'Mobile',
                     'x38':'Internet',
                     'x39':'GymMonth',
                     'x40':'Tennis1h',
                     'x41':'Cinema',
                     'x42':'PreschoolMonth',
                     'x43':'IntlPrimarySchoolYear',
                     'x44':'Jeans',
                     'x45':'SummerDress',
                     'x46':'NikeShoes',
                     'x47':'LeatherShoes',
                     'x48':'Apartment_1BR_C',
                     'x49':'Apartment_1BR_OC',
                     'x50':'Apartment_3BR_C',
                     'x51':'Apartment_3BR_OC',
                     'x52':'SquareMeter_C',
                     'x53':'SquareMeter_OC',
                     'x54':'Salary',
                     'x55':'Mortgage_IR',

                     }, 
                    inplace = True)
df = df.drop('data_quality',axis=1)

#Description of the dataset
df.describe()

### Prepare Data 

1. Group cities

    We will base our analysis on the living cost by countries. We create a new data frame grouping countries by averaging the values of their corresponding cities.

In [ ]:
df_countries = df.groupby(['country']).mean()
df_countries.reset_index(inplace=True)
df_countries.head()


2. Missing value treatment

    - Let's see how many missing values are remaining and their proportions

In [ ]:
prop_countries = (df_countries.count()/df_countries.shape[0]).sort_values()
n_mv = df_countries.isna().sum().sum()

print('The proportion of missing values for every column is: ')
print(prop_countries)

print('In total, there are {} missing values in the dataframe'.format(n_mv))


- We fill the 9 missing values in the dataframe with the most common value of the column for each indicator.

In [ ]:
df_countries_2 = df_countries.select_dtypes(include='float')

fill_mode = lambda col: col.fillna(col.mode()[0])

new_df = df_countries_2.apply(fill_mode, axis=1)
n_mv = new_df.isna().sum().sum()

print('Number of missing values now: {}'.format(n_mv))

df_countries = pd.concat([df_countries['country'],new_df],axis=1)

### Analysis with visualization

We use the geopandas library to visualize in a worldwide map the indicators of each country. Parts of the following code are taken from: https://www.kaggle.com/code/lauman/data-visualization-on-cost-of-living-dataset


In [ ]:
import geopandas as gpd  
SHAPEFILE = 'ne_10m_admin_0_countries.shp'
geo_df = gpd.read_file(SHAPEFILE)

geo_df['ADMIN'] = geo_df['ADMIN'].replace({
    'United States of America': 'United States',
    'Hong Kong S.A.R.': 'Hong Kong',
    'United States of Tanzania': 'Tanzania',
    'Republic of Serbia': 'Serbia'})

geo_df = geo_df[['ADMIN', 'geometry']].set_index('ADMIN')


Let's take a look on the average salary of each country on the map

In [ ]:
import plotly.figure_factory as ff 
import plotly.express as px 

col = 'Salary'

fig = px.choropleth_mapbox(df_countries, geojson=geo_df.geometry, locations='country', color=col,
                           color_continuous_scale="Viridis",
                           range_color=(df_countries[col].min(), df[col].max()),
                           mapbox_style="carto-positron",
                           zoom=0, center={"lat": 22.3193, "lon": 114.1694},
                           opacity=0.5,
                           labels={col:'Average Monthly Net Salary (After Tax) (USD)'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()



It is noticable that salaries are higher in North America and Europe than in Latin America, Asia, and Africa. However, living costs may be also higher in the countries of these continents. Let's take a look to some indicators.

Note: In the following plot, it is recommended to zoom in to get a better picture of the data distribution.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def indicators_subplot(df_countries, indicators):
    '''  Plot the indicators' cost of each country in descending order

    Args:
        df_countries (DataFrame): Dataframe that contains data about countries

    '''
    x = df_countries['country'] 
    
    fig = make_subplots(rows=2, cols=2)

    traces = []
    for i in range(4):
        traces.append(go.Bar(x=x, y=df_countries[indicators[i]],name=indicators[i])) 

    #Add traces to the subplot
    fig.append_trace(traces[0], 1, 1)
    fig.append_trace(traces[1], 1, 2)
    fig.append_trace(traces[2], 2, 1)
    fig.append_trace(traces[3], 2, 2)
    #Overall figure config
    fig.update_xaxes(categoryorder='total descending')
    fig.update_layout(height=1000, width=1000,title='Overview of some cost indicators among countries')
    fig.show()

indicators = ['MealInexpensive', 'Gasoline','Services','SquareMeter_C']
indicators_subplot(df_countries, indicators)

It seems that countries located at the right tail of the barplots are located in Latin America and Africa, which suggests that even though these people earn less, they may spend less also. In order to understand the relation between salary and costs in each country, a proportion should be made.

Let's create a dataframe that contains the salary-cost ratio for each indicator and visualize some  of them on the world map.

In [ ]:

df_indicators = df_countries.drop(['country','Salary'],axis=1)
ratio_df = df_indicators.divide(df_countries['Salary'],axis=0)
ratio_df = pd.concat([df_countries['country'],ratio_df],axis=1)


Let's visualize the salary-cost ratio for the same previous indicators: MealInexpensive, Gasoline, Services, and Square_Meter_C

In [ ]:
col = 'MealInexpensive'

fig = px.choropleth_mapbox(ratio_df, geojson=geo_df.geometry, locations='country', color=col,
                           color_continuous_scale="Turbo",
                           range_color=(ratio_df[col].max(), ratio_df[col].min()),
                           mapbox_style="carto-positron",
                           zoom=0, center={"lat": 22.3193, "lon": 114.1694},
                           opacity=0.5,
                           labels={col:'Meal in an Inexpensive Restaurant and Salary ratio'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
col = 'Gasoline'

fig = px.choropleth_mapbox(ratio_df, geojson=geo_df.geometry, locations='country', color=col,
                           color_continuous_scale="Turbo",
                           range_color=(ratio_df[col].max(), ratio_df[col].min()),
                           mapbox_style="carto-positron",
                           zoom=0, center={"lat": 22.3193, "lon": 114.1694},
                           opacity=0.5,
                           labels={col:'Gasoline and Salary ratio'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
col = 'Services'

fig = px.choropleth_mapbox(ratio_df, geojson=geo_df.geometry, locations='country', color=col,
                           color_continuous_scale="Turbo",
                           range_color=(ratio_df[col].max(), ratio_df[col].min()),
                           mapbox_style="carto-positron",
                           zoom=0, center={"lat": 22.3193, "lon": 114.1694},
                           opacity=0.5,
                           labels={col:'Services and Salary ratio'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
col = 'SquareMeter_C'

fig = px.choropleth_mapbox(ratio_df, geojson=geo_df.geometry, locations='country', color=col,
                           color_continuous_scale="Turbo",
                           range_color=(ratio_df[col].min(), ratio_df[col].max()),
                           mapbox_style="carto-positron",
                           zoom=0, center={"lat": 22.3193, "lon": 114.1694},
                           opacity=0.5,
                           labels={col:'Price per Square Meter to Buy Apartment in City Centre and Salary Ratio'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

From the visualizations, it is possible to note that there is a great difference throughout the countries in terms of the proportion of the salary that must be used to pay for the expenses of the indicators studied. In countries like Cuba, people need their entire salary to pay for basic services, while in United States and Canada, this only represents 4% of their salary. 

The 5 countries with highest indicator salary ratio are shown in the tables below. 

In [ ]:
top_meal = ratio_df.sort_values(by=['MealInexpensive'], axis=0, ascending=False)[['country','MealInexpensive']]
top_meal.head()

In [ ]:
top_meal = ratio_df.sort_values(by=['Gasoline'], axis=0, ascending=False)[['country','Gasoline']]
top_meal.head()

In [ ]:
top_meal = ratio_df.sort_values(by=['Services'], axis=0, ascending=False)[['country','Services']]
top_meal.head()

In [ ]:
top_meal = ratio_df.sort_values(by=['SquareMeter_C'], axis=0, ascending=False)[['country','SquareMeter_C']]
top_meal.head()

- What are the countries with highest and lowest Salary-Cost ratio considering the following indicators?

Cuba tops the list in 3/4 indicators. Considering these four indicators, we can say that the monthly salary in Cuba is not enough to fulfill the average living costs. Also, most of the remaining countries of the list are african countries, which is an important social interest matter to consider globally.

- The next question that we would like to answer is: Do countries with highest Cost of Living Index are necessarly the ones with highest purchasing power index?



To answer this question we want to not only consider 4 indicators, but all the indicators available. The way to achieve this is to calculate a Cost of Living Index by computing a weighted average between all indicators. To obtain the purchasing power index, we divide the average salary of each country by their cost of living index. 

We will consider the weights that are adopted by Numbeo (https://www.numbeo.com/common/motivation_and_methodology.jsp). 

Note that Mobile weight is considerably high because that is the cost for only one minute of prepaid mobile.

In [ ]:
df_weights = pd.DataFrame([['MealInexpensive',16],
                     ['Meal2People',3.5],
                     ['McMeal',6],
                     ['BeerDomesticRes',5],
                     ['BeerImportedRes',5],
                     ['Cappuccino',15],
                     ['Coke',6],
                     ['Water',6],
                     ['Milk',25],
                     ['Bread',31],
                     ['Rice',14],
                     ['Eggs',20],
                     ['Cheese',12],
                     ['Chicken',15],
                     ['Beef',15],
                     ['Apples',31],
                     ['Banana',25],
                     ['Oranges',30],
                     ['Tomato',22],
                     ['Potato',24],
                     ['Onion',10],
                     ['Lettuce',18],
                     ['WaterM',30],
                     ['Wine',4],
                     ['BeerDomesticMarket',6],
                     ['BeerImportedMarket',6],
                     ['Cigarettes',15],
                     ['OWTicket',20],
                     ['MonthlyPass',1.5],
                     ['TaxiStart',5],
                     ['Taxi1km',20],
                     ['Taxi1h',1.5],
                     ['Gasoline',60],
                     ['VWGolf',0.0035],
                     ['Corolla',0.0035],
                     ['Services',1],
                     ['Mobile',320],
                     ['Internet',1],
                     ['GymMonth',2.3],
                     ['Tennis1h',3],
                     ['Cinema',6],
                     ['PreschoolMonth',0],
                     ['IntlPrimarySchoolYear',0],
                     ['Jeans',0.35],
                     ['SummerDress',0.35],
                     ['NikeShoes',0.35],
                     ['LeatherShoes',0.35],
                     ['Apartment_1BR_C',0.25],
                     ['Apartment_1BR_OC',0.25],
                     ['Apartment_3BR_C',0.25],
                     ['Apartment_3BR_OC',0.25],
                     ['SquareMeter_C',0],
                     ['SquareMeter_OC',0],
                     ['Salary',0],
                     ['Mortgage_IR',0]],columns=['Indicator','Weight']
                     )

fig = px.bar(df_weights.sort_values(by=['Weight'],ascending=False), x='Indicator', y=['Weight'],opacity=0.8)

fig.show()

In [ ]:
p=df_countries.drop(['country'],axis=1)
row = list(df_weights['Weight'].T)
df_countries['Index'] = p.mul(row,axis=1).sum(axis=1)
df_countries['Power_index'] = df_countries['Salary'].divide(df_countries['Index'],axis=0) 


In [ ]:

x=df_countries['country']

fig = make_subplots(rows=2, cols=1)

trace0 = go.Bar(x=x, y=df_countries['Index'],name='Cost of Living Index')
trace1 = go.Bar(x=x, y=df_countries['Power_index'],name='Purchasing Power Index')

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 2, 1)

fig.update_xaxes(categoryorder='total descending')
fig.update_layout(yaxis_title=("Cost (USD)"))
fig.show()



- Do countries with highest Cost of Living Index are necessarly the ones with highest purchasing power index?

The countries of highest Cost of Living Index are: Bermuda, Switzerland, Singapore, Hong Kong, and Jersey. However, the countries with highest purchasing power index are: Isle of Man, United States, Switzerland, Australia, and New Zaeland. 

Countries like Bermuda, for example, have a considerably high cost of living, but the purchasing power index is average (around 0.5). This means that people do not earn enough money to afford the costs of their lives in that country. 

Inspecting both graphs closely, it is clear that people that live in some high cost of living countries do not earn enough money to have also a high purchasing power. Hence, countries with highest Cost of Living Index are not necessarly the ones with highest purchasing power index

### Modelling

The next question is to determine if we can predict the average monthly salary of a country by training a model with their living costs indicators. For this, we will train a Random Forest Regressor and evaluate it with the R2 score. 

In [ ]:
from sklearn.model_selection import train_test_split

X = df_countries.drop(['country','Salary','Power_index'],axis=1)

y = df_countries['Salary']

# Split data into training and test data, and fit a linear model
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=.20, random_state=44)

from sklearn.ensemble import RandomForestRegressor

#Initialize Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=30, max_features="auto", random_state=44, max_depth=20)
#Fit the model 
rf_model.fit(X_train, y_train)
#Predict on the test split
predictions = rf_model.predict(X_test)
#Evaluate the model
r2_test = r2_score(y_test, predictions) * 100

print("The model's score is {:.2f} %".format(r2_test))


The model's score is 86.65%, which means that it is possible to predict the average monthly salary of a country based only on their costs. This is a powerful tool because it means you may customize costs and predict the salary based on them.

### Evaluation and Findings

According to the defined indicators, there is a huge difference between countries in terms of the proportion of the salary that must be used to pay for the expenses of the indicators studied. 

Also, we found the countries with most and least purchasing power index, which is a very important indicator of quality of life.

Finally, we trained a Random Forest Regressor to predict the average salary of a country based on their costs of living. This model is useful for predicting salary of modified costs of living and for filling missed values in other datasets.